In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%watermark -v -p tensorflow,tensorboard
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.5
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
CPython 3.6.9
IPython 7.16.1

tensorflow 2.6.0
tensorboard 2.6.0


In [2]:
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'tensorboard-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))

@register_line_cell_magic
def template_writefile(line, cell):
    with open(line, 'w') as fw:
        fw.write(cell.format(**globals()))

@register_line_magic
def netron(line):
    args = line.split()
    file, port, height = args[0], int(args[1]), 600
    if len(args) == 3:
        height = int(args[2])
    # res = !lsof -i:$port | grep $port
    # if len(res) == 1:
    #     pid = int(res[0].split(' ')[1])
    #     !kill -9 $pid
    import netron
    netron.start(file, address=('0.0.0.0', port), browse=False)
    display_html(port, height)


In [3]:
import tensorflow as tf

layers = tf.keras.layers
regularizers = tf.keras.regularizers

def pairwise_l2_distance(a, b):
  """Computes pairwise distances between all rows of a and all rows of b."""
  norm_a = tf.reduce_sum(tf.square(a), 1)
  norm_a = tf.reshape(norm_a, [-1, 1])
  norm_b = tf.reduce_sum(tf.square(b), 1)
  norm_b = tf.reshape(norm_b, [1, -1])
  dist = tf.maximum(norm_a - 2.0 * tf.matmul(a, b, False, True) + norm_b, 0.0)
  return dist


def get_sims(embs, temperature):
  """Calculates self-similarity between batch of sequence of embeddings."""
  batch_size = tf.shape(embs)[0]
  seq_len = tf.shape(embs)[1]
  embs = tf.reshape(embs, [batch_size, seq_len, -1])

  def _get_sims(embs):
    """Calculates self-similarity between sequence of embeddings."""
    dist = pairwise_l2_distance(embs, embs)
    sims = -1.0 * dist
    return sims

  sims = tf.map_fn(_get_sims, embs)
  sims /= temperature
  sims = tf.nn.softmax(sims, axis=-1)
  sims = tf.expand_dims(sims, -1)
  return sims

# (1, 64, 64, 32)
#     x = tf.reshape(x, [batch_size, self.num_frames, -1])
# (1, 64, 2048)
# tf.keras.layers.Reshape((1, 64, -1)),
class TestModel(tf.keras.models.Model):
    def __init__(self): 
        super(TestModel, self).__init__()
        base_model = tf.keras.applications.ResNet50V2(
            include_top=False, weights=None, pooling='max')
        self.base_model = tf.keras.models.Model(
            inputs=base_model.input,
            outputs=base_model.get_layer('conv4_block3_out').output)
        
        self.temporal_conv_layers = [
            layers.Conv3D(512,
                          3,
                          padding='same',
                          dilation_rate=(3, 1, 1),
                          kernel_regularizer=regularizers.l2(1e-6),
                          kernel_initializer='he_normal')]
            
        self.temporal_bn_layers = [layers.BatchNormalization()
                               for _ in self.temporal_conv_layers]

        self.conv_3x3_layer = layers.Conv2D(32,
                                            3,
                                            padding='same',
                                            activation=tf.nn.relu)
        self.input_projection = layers.Dense(
            512, kernel_regularizer=regularizers.l2(1e-6),
            activation=None, name='RE_input_projection')


    def call(self, x):
        print('input:', x.shape)
        batch_size = tf.shape(x)[0]
        x = tf.reshape(x, [-1, 112, 112, 3])
        print('before base_model', x.shape)
        x = self.base_model(x)
        print('after base_model', x.shape)
        h = tf.shape(x)[1]
        w = tf.shape(x)[2]
        c = tf.shape(x)[3]
        x = tf.reshape(x, [batch_size, -1, h, w, c])
        print('reshape:', x.shape)
            
        for bn_layer, conv_layer in zip(self.temporal_bn_layers,
                                        self.temporal_conv_layers):
            x = conv_layer(x)
            x = bn_layer(x)
            x = tf.nn.relu(x)
            
        print('temporal:', x.shape)
        x = tf.reduce_max(x, [2, 3])
        print('reduce max:', x.shape)
        
        x = get_sims(x, 13.45)
        print('sims x:', x.shape)
        
        x = self.conv_3x3_layer(x)
        print('before reshape: ', x.shape)
        x = tf.reshape(x, [batch_size, 64, -1])
        print('after reshape', x.shape)
        
        x = self.input_projection(x)
        print('input_projection:', x.shape)

        return x
        
model1 = TestModel()
model1(tf.random.uniform((1, 64, 112, 112, 3)));

# model1 = tf.keras.Sequential(
#     [
#         tf.keras.layers.Dense(512, activation=None),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Dense(10, activation="softmax"),
#     ]
# )
# 
# @tf.function
# def traceme(x):
#     return model1(x)
# 
# 
# logdir = "log2"
# writer = tf.summary.create_file_writer(logdir)
# tf.summary.trace_on(graph=True, profiler=True)
# # Forward pass
# traceme(tf.random.uniform((1, 64, 2048)))
# with writer.as_default():
#     tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=logdir)
    
os.system('rm -rf /data/ff')
model1.save('/data/ff', save_format='tf')

input: (1, 64, 112, 112, 3)
before base_model (64, 112, 112, 3)
after base_model (64, 7, 7, 1024)
reshape: (1, 64, 7, 7, 1024)
temporal: (1, 64, 7, 7, 512)
reduce max: (1, 64, 512)
sims x: (1, 64, 64, 1)
before reshape:  (1, 64, 64, 32)
after reshape (1, 64, 2048)
input_projection: (1, 64, 512)
input: (None, 64, 112, 112, 3)
before base_model (None, 112, 112, 3)
after base_model (None, 7, 7, 1024)
reshape: (None, None, None, None, None)
temporal: (None, None, None, None, 512)
reduce max: (None, None, 512)
sims x: (None, None, None, 1)
before reshape:  (None, None, None, 32)
after reshape (None, 64, None)
input_projection: (None, 64, 512)
input: (None, 64, 112, 112, 3)
before base_model (None, 112, 112, 3)
after base_model (None, 7, 7, 1024)
reshape: (None, None, None, None, None)
temporal: (None, None, None, None, 512)
reduce max: (None, None, 512)
sims x: (None, None, None, 1)
before reshape:  (None, None, None, 32)
after reshape (None, 64, None)
input_projection: (None, 64, 512)
inpu

/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
!python3 -m tf2onnx.convert --tag serve --signature_def serving_default --opset 13 \
    --saved-model /data/ff --output /data/ff/test.onnx --verbose --debug

In [ ]:
%netron /data/ff/test.onnx 8222